In [1]:
import pyspark
from pyspark.sql import SparkSession
import os
from dotenv import load_dotenv

load_dotenv()

NESSIE_URI = os.environ.get("NESSIE_URI") 

spark = (
    SparkSession.builder
        .appName("Iceberg-Nessie-rest-catalog")
        .master("local[*]")
        .config("spark.driver.host", "127.0.0.1")
        .config("spark.jars.packages", ",".join([
            "org.projectnessie.nessie-integrations:nessie-spark-extensions-3.5_2.12:0.103.2",
            "org.apache.iceberg:iceberg-spark-runtime-3.5_2.12:1.8.1",
            "org.apache.iceberg:iceberg-aws-bundle:1.8.1"
        ]))
        .config("spark.sql.extensions", "org.projectnessie.spark.extensions.NessieSparkSessionExtensions,org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions")
        .config("spark.sql.catalog.nessie", "org.apache.iceberg.spark.SparkCatalog")
        .config("spark.sql.catalog.nessie.type", "rest")
        .config("spark.sql.catalog.nessie.uri", NESSIE_URI)
        .config("spark.sql.catalogImplementation", "in-memory")
        .config("spark.sql.catalog.nessie.scope", "catalog sign")
        .getOrCreate()
)

:: loading settings :: url = jar:file:/home/phamminh/work/BIDV/kedro_research/kedro-spark/.venv/lib/python3.12/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/phamminh/.ivy2/cache
The jars for the packages stored in: /home/phamminh/.ivy2/jars
org.projectnessie.nessie-integrations#nessie-spark-extensions-3.5_2.12 added as a dependency
org.apache.iceberg#iceberg-spark-runtime-3.5_2.12 added as a dependency
org.apache.iceberg#iceberg-aws-bundle added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-5f60c47b-3a8f-4110-97a8-089ca53a8bd3;1.0
	confs: [default]
	found org.projectnessie.nessie-integrations#nessie-spark-extensions-3.5_2.12;0.103.2 in central
	found org.apache.iceberg#iceberg-spark-runtime-3.5_2.12;1.8.1 in central
	found org.apache.iceberg#iceberg-aws-bundle;1.8.1 in central
:: resolution report :: resolve 225ms :: artifacts dl 14ms
	:: modules in use:
	org.apache.iceberg#iceberg-aws-bundle;1.8.1 from central in [default]
	org.apache.iceberg#iceberg-spark-runtime-3.5_2.12;1.8.1 from central in [default]
	org.projectnessie.nessie-integrations#nessie-spark-extensions-3.5_2

In [2]:
spark.sql("SHOW DATABASES IN nessie").show()

+----------+
| namespace|
+----------+
|kedro_demo|
+----------+



In [8]:
spark.sql("SHOW tables IN nessie.kedro_demo").show()

+----------+---------+-----------+
| namespace|tableName|isTemporary|
+----------+---------+-----------+
|kedro_demo|  reviews|      false|
|kedro_demo|     test|      false|
+----------+---------+-----------+



In [7]:
spark.catalog.tableExists("kedro_demo.reviews")

False

In [10]:
spark.sql(f"DESCRIBE TABLE nessie.kedro_demo.reviews1").collect()

AnalysisException: [TABLE_OR_VIEW_NOT_FOUND] The table or view `nessie`.`kedro_demo`.`reviews1` cannot be found. Verify the spelling and correctness of the schema and catalog.
If you did not qualify the name with a schema, verify the current_schema() output, or qualify the name with the correct schema and catalog.
To tolerate the error on drop use DROP VIEW IF EXISTS or DROP TABLE IF EXISTS.; line 1 pos 15;
'DescribeRelation false, [col_name#66, data_type#67, comment#68]
+- 'UnresolvedTableOrView [nessie, kedro_demo, reviews1], DESCRIBE TABLE, true


In [4]:
spark.sql("CREATE DATABASE IF NOT EXISTS nessie.demo_haha").show()

++
||
++
++



In [3]:
spark.sql(
    """
    CREATE TABLE IF NOT EXISTS nessie.demo_haha.sample_table (
        id BIGINT,
        name STRING
    ) USING iceberg
    """
).show()

++
||
++
++



In [4]:
spark.sql(
    """
    INSERT INTO nessie.demo_haha.sample_table VALUES
    (3, 'Nguuu'),
    (4, 'Lonnn')
    """
).show()

++
||
++
++



In [3]:
spark.sql("SELECT * FROM  nessie.demo_haha.sample_table;").show()

+---+-----+
| id| name|
+---+-----+
|  1|Alice|
|  1|Alice|
|  2|  Bob|
|  2|  Bob|
|  3|Nguuu|
|  4|Lonnn|
+---+-----+

